In [93]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
financial_dir={}
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [94]:
# tickers=['ADANIPORTS.NS','ASIANPAINT.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS','BAJAJFINSV.NS','BPCL.NS','BHARTIARTL.NS','INFRATEL.NS','CIPLA.NS','COALINDIA.NS','DRREDDY.NS','EICHERMOT.NS','GAIL.NS','GRASIM.NS','HCLTECH.NS','HDFCBANK.NS','HEROMOTOCO.NS','HINDALCO.NS','HINDPETRO.NS','HINDUNILVR.NS','HDFC.NS','ITC.NS','ICICIBANK.NS','IBULHSGFIN.NS','IOC.NS','INDUSINDBK.NS','INFY.NS','JSWSTEEL.NS','KOTAKBANK.NS','LT.NS','M&M.NS','MARUTI.NS','NTPC.NS','ONGC.NS','POWERGRID.NS','RELIANCE.NS','SBIN.NS','SUNPHARMA.NS','TCS.NS','TATAMOTORS.NS','TATASTEEL.NS','TECHM.NS','TITAN.NS','UPL.NS','ULTRACEMCO.NS','VEDL.NS','WIPRO.NS','YESBANK.NS','ZEEL.NS']

tickers = ["AXP","AAPL","BA","CAT","CVX","CSCO","DIS","DOW", "XOM","HD","IBM","INTC","JNJ","KO","MCD","MMM","MRK","MSFT","NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT","WBA"]



In [95]:
for ticker in tickers:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c)"}) # try soup.findAll("table") if this line gives error 
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        
        #combining all extracted information with the corresponding ticker
        financial_dir[ticker] = temp_dir
    except:
        print("Problem scraping data for ",ticker)


#storing information in pandas dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted
for ticker in tickers:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]


In [96]:
combined_financials

,ADANIPORTS.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BPCL.NS,BHARTIARTL.NS,INFRATEL.NS,CIPLA.NS,COALINDIA.NS,DRREDDY.NS,EICHERMOT.NS,GAIL.NS,GRASIM.NS,HCLTECH.NS,HDFCBANK.NS,HEROMOTOCO.NS,HINDALCO.NS,HINDPETRO.NS,HINDUNILVR.NS,HDFC.NS,ITC.NS,ICICIBANK.NS,IBULHSGFIN.NS,IOC.NS,INDUSINDBK.NS,INFY.NS,JSWSTEEL.NS,KOTAKBANK.NS,LT.NS,M&M.NS,MARUTI.NS,NTPC.NS,ONGC.NS,POWERGRID.NS,SBIN.NS,SUNPHARMA.NS,TCS.NS,TATAMOTORS.NS,TECHM.NS,TITAN.NS,UPL.NS,ULTRACEMCO.NS,VEDL.NS,YESBANK.NS,ZEEL.NS
Cash,"7,19,54,600","56,38,300",NaN,"28,54,000",NaN,NaN,"41,42,500","13,55,07,000","14,52,000","50,83,600","2,27,57,100","20,53,000","4,32,100","29,67,200","1,22,39,400","8,53,717",NaN,"37,85,300","9,11,87,400","19,87,400","3,21,60,000",NaN,"31,78,100",NaN,NaN,"93,32,800",NaN,"24,65,000","3,96,60,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash and cash equivalents,"4,79,81,900","44,48,800",NaN,"91,56,400",NaN,NaN,"1,35,38,600","6,21,21,000","2,000","85,34,600","4,19,26,700","22,28,000","72,69,800","1,37,73,300","94,96,400","2,60,198",NaN,"24,30,400","8,04,49,400","1,30,51,800","62,10,000",NaN,"15,30,700",NaN,NaN,"31,89,000",NaN,"28,29,000","5,58,10,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other short-term investments,"1,58,13,900","1,35,23,100",NaN,"1,57,64,800",NaN,NaN,"5,44,92,800","4,65,05,000","2,95,63,000","1,11,22,900","27,46,53,700","2,25,29,000","2,69,48,600","1,15,12,000","8,85,37,200","3,62,035",NaN,"3,17,38,800","3,90,44,700","4,99,93,800","6,23,30,000",NaN,"13,15,71,500",NaN,NaN,"9,38,80,200",NaN,"9,58,000","79,20,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total cash,72.26B,17.38B,518.02B,30.65B,167.37B,284.33B,131.14B,480.07B,27.28B,25.43B,322.52B,27.69B,54.5B,20.16B,170.05B,1.34B,341.4B,50.14B,310.4B,135.36B,74.37B,1.91T,270.44B,323.42B,157.78B,247.58B,160.54B,2.99B,121.61B,562.96B,259.01B,147.52B,17.55B,162.42B,265.68B,112.57B,1.08T,123.14B,393.42B,491.75B,86.86B,2.78B,75.53B,46.51B,395.66B,41.17B,13.85B
Net receivables,"2,78,96,600","1,90,73,300",NaN,"2,55,96,900",NaN,NaN,"5,20,47,900","4,30,06,000","55,09,000","3,10,24,500","8,68,91,600","3,98,69,000","9,03,400","3,42,95,600","5,21,31,400","14,80,550",NaN,"2,74,51,100","9,95,98,100","5,58,70,200","1,81,60,000",NaN,"2,68,22,900",NaN,NaN,"10,69,64,800",NaN,"21,44,000","7,16,00,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Inventory,"-4,46,500","-25,05,300",NaN,"-10,19,900",NaN,NaN,"-40,39,300","-5,22,000",NaN,"10,38,400","85,99,200","-14,87,000","6,10,300","-57,04,300","-58,01,700","15,551",NaN,"-3,27,900","-91,000","-1,83,33,500","-33,10,000",NaN,"-35,94,400",NaN,NaN,"-6,58,00,100",NaN,NaN,"74,40,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other current assets,"17,37,300",100,NaN,"2,400",NaN,NaN,"3,86,00,600","28,71,000","23,000","11,200",-,"48,24,000","8,99,100",-,"1,16,00,100","1,382",NaN,NaN,NaN,"11,77,300","10,000",NaN,"8,94,400",NaN,NaN,"2,900",NaN,"32,000","8,70,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total current assets,"14,63,11,400","7,63,40,500",NaN,"7,07,31,600",NaN,NaN,"40,59,92,800","32,90,57,000","4,28,00,000","10,81,41,000","68,81,42,800","11,45,01,000","4,41,02,700","10,88,84,200","40,24,88,500","37,72,109",NaN,"8,41,32,100","49,85,43,700","37,14,09,600","11,91,40,000",NaN,"26,39,36,200",NaN,NaN,"1,05,67,44,500",NaN,"76,44,000","33,55,50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Property, plant and equipment","36,79,51,500","7,62,80,200",NaN,"4,06,78,200",NaN,NaN,"66,45,07,500","2,32,10,18,000","17,12,17,000","7,83,69,800","53,91,27,700","11,56,43,000","3,31,07,200","47,55,06,000","61,06,96,400","15,20,678",NaN,"9,97,62,400","1,07,21,50,800","61,74,54,400","7,99,40,000",NaN,"27,29,84,200",NaN,NaN,"1,80,88,86,300",NaN,"41,06,000","1,06,64,50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [97]:
tickers=combined_financials.columns
stats = ["EBITDA",
         "Depreciation & amortisation",
         "Market cap (intra-day)",
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt",
         "Forward annual dividend yield"] 

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]


In [98]:
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)
            

In [99]:
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
   

In [100]:
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)
tickers = all_stats_df.columns

In [101]:
all_stats_df

,ADANIPORTS.NS,ASIANPAINT.NS,BAJAJ-AUTO.NS,BPCL.NS,BHARTIARTL.NS,CIPLA.NS,COALINDIA.NS,DRREDDY.NS,GAIL.NS,GRASIM.NS,HCLTECH.NS,HEROMOTOCO.NS,HINDALCO.NS,HINDPETRO.NS,ITC.NS,IOC.NS,JSWSTEEL.NS
EBITDA,7.033000e+07,3.282000e+07,5.788000e+07,9.356000e+07,3.441000e+08,3.206000e+07,2.195000e+08,4.156000e+07,9.025000e+07,1.746700e+08,2.460000e+06,3.904000e+07,1.440600e+08,6.538000e+07,1.730100e+08,1.686000e+08,1.004100e+08
D&A,1.680280e+07,7.819400e+06,2.464300e+06,3.417770e+07,2.768960e+08,1.326310e+07,3.450360e+07,1.247200e+07,1.666640e+07,3.260450e+07,2.984910e+05,8.457600e+06,4.776980e+07,3.085310e+07,1.396610e+07,8.507380e+07,4.246000e+07
MarketCap,6.404100e+08,1.640000e+09,8.684100e+08,8.974400e+08,3.030000e+09,5.806600e+08,7.965300e+08,7.512000e+08,4.361300e+08,4.164200e+08,1.910000e+09,5.346100e+08,3.637100e+08,3.276200e+08,2.390000e+09,8.326800e+08,5.290100e+08
NetIncome,3.763130e+07,2.705170e+07,5.211910e+07,7.802300e+07,-3.218320e+08,1.527700e+07,1.746185e+08,1.949800e+07,6.545740e+07,1.771920e+07,1.457179e+06,3.638110e+07,5.495670e+07,6.690630e+07,1.259233e+08,1.737670e+08,4.030000e+07
CashFlowOps,7.401810e+07,2.631890e+07,3.850440e+07,1.015715e+08,1.812870e+08,1.691140e+07,1.669546e+08,2.984100e+07,7.984200e+07,-2.454820e+07,1.291734e+06,5.518130e+07,1.197952e+08,8.554030e+07,1.258341e+08,1.413304e+08,1.278500e+08
Capex,-3.621410e+07,-4.136900e+06,-2.967400e+06,-1.054130e+08,-2.071680e+08,-5.271400e+06,-7.339320e+07,-6.115000e+06,-7.795810e+07,-4.083440e+07,-4.989250e+05,-1.287800e+07,-6.005340e+07,-1.133802e+08,-3.169120e+07,-2.688121e+08,-1.281000e+08
CurrAsset,1.463114e+08,7.634050e+07,7.073160e+07,4.059928e+08,3.290570e+08,1.081410e+08,6.881428e+08,1.145010e+08,1.088842e+08,4.024885e+08,3.772109e+06,8.413210e+07,4.985437e+08,3.714096e+08,2.639362e+08,1.056744e+09,3.355500e+08
CurrLiab,1.035169e+08,5.028870e+07,4.873780e+07,4.570185e+08,9.305490e+08,3.832620e+07,5.079508e+08,5.970000e+07,1.118630e+08,4.229193e+08,1.552435e+06,4.409180e+07,3.231652e+08,4.756445e+08,9.250150e+07,1.398641e+09,4.200800e+08
PPE,2.726441e+08,5.465850e+07,1.700230e+07,4.980699e+08,9.036610e+08,5.827700e+07,3.786085e+08,5.408800e+07,3.521511e+08,4.994982e+08,7.495680e+05,5.177350e+07,6.586957e+08,4.221470e+08,2.136328e+08,1.382735e+09,7.314400e+08
BookValue,2.453820e+08,9.519690e+07,2.323381e+08,3.661857e+08,7.142220e+08,1.422919e+08,1.984657e+08,1.401970e+08,4.167857e+08,5.736185e+08,5.588889e+06,1.312041e+08,5.485177e+08,2.553244e+08,5.251011e+08,1.138738e+09,3.479500e+08


In [102]:
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]





In [103]:

# finding value stocks based on Magic Formula
final_stats_val_df = final_stats_df.loc[tickers,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)


# finding highest dividend yield stocks
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)


# # Magic Formula & Dividend yield combined
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)

------------------------------------------------
Value stocks based on Greenblatt's Magic Formula
               EarningYield       ROC  MagicFormulaRank
COALINDIA.NS       0.295106  0.331060               1.0
BAJAJ-AUTO.NS      0.065375  1.421057               2.0
ITC.NS             0.071688  0.413029               3.0
GRASIM.NS          0.168172  0.296546               4.0
GAIL.NS            0.163415  0.210737               5.0
HEROMOTOCO.NS      0.061793  0.333092               6.0
ADANIPORTS.NS      0.067207  0.169691               7.0
HINDALCO.NS        0.144515  0.115446               8.0
HCLTECH.NS         0.001133  0.727967               9.0
ASIANPAINT.NS      0.015489  0.309757              10.0
DRREDDY.NS         0.040521  0.267134              11.0
HINDPETRO.NS       0.065341  0.108605              12.0
BPCL.NS            0.047985  0.132833              13.0
BHARTIARTL.NS      0.014921  0.222405              14.0
JSWSTEEL.NS        0.064884  0.089580              15.0
CIPLA.